In [1]:
import os
import gzip
import pandas as pd
from io import StringIO
import numpy as np
import datetime
import dateutil.parser as parser
def read_stock_data(data_home, data_type, venue, year, month, day, ticker, is_filter):
    '''
    从sever中读取一只股票一天的数据
    data_home:数据所在folder
    data_type:类型
    venue:交易所
    is_filter:是否进行filter操作
    '''

    path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
        month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue)  # 读数据的路径

    if (os.path.exists(path)):
        data0 = open(path, 'r')
        data1 = pd.read_csv(StringIO(data0.read()))
        data = data1.loc[:, ['time', 'volume', 'current','a1_v','a1_p','b1_v','b1_p',]]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        data['spread'] = (data['a1_p'] - data['b1_p'])/(data['a1_p'] + data['b1_p'])
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]         # 删除大于99.9%分位数的数据
            return data
    else:
        path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
            month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue) + '.gz'
        with gzip.open(path, 'rb') as gf:
            data1 = pd.read_csv(gf)
        data = data1.loc[:, ['time', 'volume','current', 'a1_v','b1_v']]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]  # 删除大于99.9%分位数的数据
            return data

In [2]:
data_home = '/volume1/sinoalgo/data/sinoalgo/JQMarketData'
data_type = 'STOCK'
venue= 'XSHE'
ticker = '000725'
start_date = "2020-09-01"
end_date = "2021-06-30"
read_stock_data(data_home, data_type, venue, '2020', '09', '05', ticker, 0)
data = read_stock_data(data_home, data_type, venue, '2020', '09', '05', ticker, 0)
data.to_csv("05.000725",index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/volume1/sinoalgo/data/sinoalgo/JQMarketData/STOCK/XSHE/2020/09/05/000725.XSHE.gz'

In [3]:
pip install pandas_market_calendars

DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas_market_calendars as mcal
data_home = '/volume1/sinoalgo/data/sinoalgo/JQMarketData'
data_type = 'STOCK'
venue= 'XSHE'
ticker = '000725'
start_date = "2020-09-01"
end_date = "2021-06-30"

# 将日期格式化为年份、月份和日份的形式
start_year, start_month, start_day = start_date.split('-')
end_year, end_month, end_day = end_date.split('-')

# 创建一个空的DataFrame来存储所有数据
all_data = pd.DataFrame()

# 获取交易日历
cal = mcal.get_calendar('SSE')
schedule = cal.schedule(start_date=start_date, end_date=end_date)

# 遍历交易日历并导出每个交易日的数据
for d in schedule.index:
    date_str = d.strftime('%Y-%m-%d')
    date = d.to_pydatetime()
    year,month,day = date_str.split('-')
    
# 判断日期是否是周末或节假日
    if cal.valid_days(date, end_date).empty:
        continue
        
    # 读取指定日期的数据
    data = read_stock_data(data_home, data_type, venue, year, month, day, ticker, 0)
    
    # 将数据追加到总数据中
    all_data = all_data.append(data)
            
# 导出所有数据到CSV文件
all_data.to_csv("000725.csv", index=False)

/tmp/ipykernel_1704333/2897547862.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1704333/2897547862.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1704333/2897547862.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1704333/2897547862.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1704333/2897547862.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat